In [ ]:
import pandas as pd


In [ ]:
# Load dataframes
df = pd.read_csv('C:/Users/DELL/Linkedin-Job-Market-Analysis-using-ML/LinkedIn Scraper/job postings 2023 24/postings.csv',encoding='latin-1')
df_A = df.sample(n=20000, random_state=42)
df_B = pd.read_csv('C:/Users/DELL/Linkedin-Job-Market-Analysis-using-ML/LinkedIn Scraper/job postings 2023 24/Domain_TD.csv', encoding='latin-1')

# Initialize a new column in df_A for the cluster
df_A['clusters'] = None
df_A['title'] = df_A['title'].astype(str).fillna('')
df_A['description'] = df_A['description'].astype(str).fillna('')

In [ ]:
# # Function to find clusters and calculate probabilities
# def find_clusters_with_probabilities(title, description, df_B):
#     cluster_matches = {}
    
#     for index, row in df_B.iterrows():
#         domain = row['Domain'].strip().lower()
#         #keywords = row['keywords'].split(',')  # assuming keywords are comma-separated

#         # Check if 'keywords' is a string; if not, treat it as an empty list
#         keywords = row['keywords'].split(',') if isinstance(row['keywords'], str) else []

#         match_count = sum(1 for keyword in keywords if keyword.strip().lower() in title.lower() or keyword.strip().lower() in description.lower())
        
#         # Check if title has more than half the words from domain
#         domain_words = domain.split()
#         half_domain_word_count = len(domain_words) / 2
#         title_word_count = len(title.split())
        
#         if title_word_count > 0 and len(set(title.lower().split()) & set(domain_words)) > half_domain_word_count:
#             # If condition is met, assign 50% to this cluster
#             cluster = f"{row['Domain']}:{row['Sub-Domain']}"
#             cluster_matches[cluster] = 50  # Initial 50%

#         if match_count > 0:
#             cluster = f"{row['Domain']}:{row['Sub-Domain']}"
#             if cluster not in cluster_matches:
#                 cluster_matches[cluster] = 0  # Initialize if not already
#             cluster_matches[cluster] += match_count  # Add keyword matches to the cluster

#     if not cluster_matches:
#         return {'Uncategorized': 100}

#     # Calculate probabilities
#     total_matches = sum(cluster_matches.values())
#     probabilities = {}

#     for cluster, count in cluster_matches.items():
#         if count >= 4:
#             probabilities[cluster] = 100.0
#         else:
#             # Adjust the total match count by deducting 50 for the already assigned 50%
#             if count > 50:  # Only use counts that are more than the initial 50%
#                 probabilities[cluster] = 50 + ((count - 50) / total_matches * 50)
#             else:
#                 probabilities[cluster] = count / total_matches * 50

#     return probabilities


def find_clusters_with_probabilities(title, description, df_B):
    cluster_matches = {}

    for index, row in df_B.iterrows():
        domain = row['Domain'].strip().lower()
        sub_domain = row['Sub-Domain'].strip().lower()
        #keywords = [keyword.strip().lower() for keyword in row['keywords'].split(',')]  # assuming keywords are comma-separated
        keywords = [keyword.strip().lower() for keyword in row['keywords'].split(',')] if isinstance(row['keywords'], str) else []

        # Match keywords in description
        description_match_count = sum(1 for keyword in keywords if keyword in description.lower())
        # Match keywords in title (checking each word in title against the keywords)
        title_word_count = len([word for word in title.lower().split() if word in keywords])

        cluster = f"{row['Domain']}:{row['Sub-Domain']}"

        # If there are matches in the title (more than 0 words in title match keywords), assign 50%
        if title_word_count > 0:
            if cluster not in cluster_matches:
                cluster_matches[cluster] = 0
            cluster_matches[cluster] += 50  # Assign 50% for title matches

        # If there are more than 4 keyword matches in the description, assign 50%
        if description_match_count >= 4:
            if cluster not in cluster_matches:
                cluster_matches[cluster] = 0
            cluster_matches[cluster] += 50  # Assign 50% for description matches

        # If the description has less than 4 keyword matches, calculate percentage
        if description_match_count < 4:
            description_percentage = (description_match_count / 4) * 50
            if cluster not in cluster_matches:
                cluster_matches[cluster] = 0
            cluster_matches[cluster] += description_percentage  # Add calculated percentage

    if not cluster_matches:
        return {'Uncategorized': 100}

    # Calculate probabilities
    total_matches = sum(cluster_matches.values())
    probabilities = {}

    for cluster, count in cluster_matches.items():
        if count >= 100:
            probabilities[cluster] = 100.0
        else:
            probabilities[cluster] = (count / total_matches) * 100.0

    return probabilities

In [ ]:
# Apply the function to each row in df_A
df_A['clusters'] = df_A.apply(lambda row: find_clusters_with_probabilities(row['title'], row['description'], df_B), axis=1)

In [ ]:
# Save the updated dataframe to a new CSV file if needed
df_A.to_csv('sample_df_A_clustered.csv', index=False)